In [1]:
import json
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
import pysos
import numpy as np
import similaripy as sim
import csv
import pickle as pkl
import pandas as pd

### Loading dataset

In [2]:
f_tracks = open(r"tracks.json") 
tracks = json.load(f_tracks)

In [3]:
f_playlists = open(r"playlists.json") 
playlists = json.load(f_playlists)

In [4]:
f_albums = open(r"albums.json") 
albums = json.load(f_albums)

In [5]:
f_artists = open(r"artists.json") 
artists = json.load(f_artists)

In [6]:
f_doc2vec = open('playlist_name_test_sim_all.dict', 'rb') 
doc2vec = pkl.load(f_doc2vec)

In [7]:
ptm_sparse = sparse.load_npz(r"ptm_csr.npz")
ptm_sparse = ptm_sparse.tolil()

In [8]:
artist_sparse = sparse.load_npz(r"artist_csr.npz")
artist_sparse = artist_sparse.tolil()
print(artist_sparse.shape)

(1000000, 295860)


In [10]:
album_sparse = sparse.load_npz(r"album_csr.npz")
album_sparse = album_sparse.tolil()

In [11]:
r = len(playlists)
c = len(tracks)
ar = len(artists)
al = len(albums)
print(ar)
print(al)

295860
734684


In [17]:
index_of_tracks = dict()
#pysos.Dict('index_track.db')
for a, b in enumerate(tracks.items()):
    uri = b[0]
    v = b[1]
    t = v[0]
    index_of_tracks[t] = uri
'''pkl_file = open('index_of_tracks.pkl', 'rb')
index_of_tracks = pkl.load(pkl_file)
pkl_file.close()'''

"pkl_file = open('index_of_tracks.pkl', 'rb')\nindex_of_tracks = pkl.load(pkl_file)\npkl_file.close()"

In [18]:
p_id = 0
index_of_playlist = dict()
for i in range(len(playlists)):
    n = playlists[i][0]
    if n in index_of_playlist:
        l = index_of_playlist[n]
        l.append(p_id)
        index_of_playlist[n] = l
    else:
        index_of_playlist[n] = [p_id]
    p_id = p_id + 1
'''pkl_file = open('index_of_playlist.pkl', 'rb')
index_of_playlists = pkl.load(pkl_file)
pkl_file.close()'''

"pkl_file = open('index_of_playlist.pkl', 'rb')\nindex_of_playlists = pkl.load(pkl_file)\npkl_file.close()"

In [19]:
print(ptm_sparse.shape)

(1000000, 2262292)


In [21]:
#top_tracks = ptm_sparse.sum(axis=0)
#top_tracks = np.sum(ptm_sparse, axis = 0)
#top_tracks = top_tracks/len(playlists)
pkl_file = open('top_tracks.pkl', 'rb')
top_tracks = pkl.load(pkl_file)
pkl_file.close()

### Setting the parameters

In [22]:
norm = 'tfidf'

pl_name_weightage = 0.1
album_boost_weightage = 0.1
artist_boost_weightage = 0.1

In [23]:
if norm == 'bm25':
    ptm_sparse = sim.normalization.bm25(ptm_sparse)
    artist_sparse = sim.normalization.bm25(artist_sparse)
    album_sparse = sim.normalization.bm25(album_sparse)
elif norm == 'bm25p':
    ptm_sparse = sim.normalization.bm25plus(ptm_sparse)
    artist_sparse = sim.normalization.bm25(artist_sparse)
    album_sparse = sim.normalization.bm25(album_sparse)
elif norm == 'tfidf':
    ptm_sparse = sim.normalization.tfidf(ptm_sparse)
    artist_sparse = sim.normalization.bm25(artist_sparse)
    album_sparse = sim.normalization.bm25(album_sparse)

### Loading test set

In [24]:
#f_test_data = open(r"D:\challenge_set.json",'r')
f_test_data = open(r"sample_test_file.json",'r')
test_data = json.load(f_test_data)

In [25]:
submission_file = open(r"submission_6.csv", 'w', newline='') 
writer = csv.writer(submission_file)
r = ['team_info', 'team_python', 'mt1180750@iitd.ac.in']
writer.writerow(r)

44

In [26]:
def get_doc2vec_score(name, doc2vec, index_of_playlist, playlists, tracks):
    pl_name_list = doc2vec[name]
    doc2vec_score = np.zeros((1,c), dtype = 'uint8')
    for j in range(min(len(pl_name_list), 5)):
        pln = pl_name_list[j]
        pli = index_of_playlist[pln]
        for k in range(len(pli)):
            ind = pli[k]
            tt = playlists[ind][1]
            for l in tt:
                t_index = tracks[l][0]
                doc2vec_score[0,t_index] = doc2vec_score[0,t_index] + 1
    return doc2vec_score
def get_vectors(track_list, tracks, albums, artists, c, ar, al):
    pl_vec = np.zeros((1,c), dtype = np.int8)
    pl_artist_vec = np.zeros((1,ar), dtype = np.int8)
    pl_album_vec = np.zeros((1,al), dtype = np.int8)
    for t in track_list:
        t_uri = t['track_uri']
        t_ar_uri = t['artist_uri']
        t_al_uri = t['album_uri']
        
        t_index = tracks[t_uri][0]
        pl_vec[0,t_index] = 1
        
        t_ar_index = artists[t_ar_uri][0]
        pl_artist_vec[0,t_ar_index] = 1
        
        t_al_index = albums[t_al_uri][0]
        pl_album_vec[0,t_al_index] = 1
        
    return pl_vec, pl_artist_vec, pl_album_vec
def get_artist_sim_score(pl_artist_vec, artist_sparse):
    artist_sim = cosine_similarity(artist_sparse,pl_artist_vec)
    return artist_sim
    #artist_sim = artist_sim.T
    #recommend = artist_sim*artist_sparse
    #recommend = np.sum(recommend, axis = 0)
    #return recommend
def get_album_sim_score(pl_album_vec, album_sparse):
    album_sim = cosine_similarity(album_sparse,pl_album_vec)
    return album_sim
    #album_sim = album_sim.T
    #recommend = album_sim*album_sparse
    #recommend = np.sum(recommend, axis = 0)
    #return recommend
def get_playlist_sim_score(pl_vec, ptm_sparse):
    pl_sim = cosine_similarity(ptm_sparse,pl_vec)
    pl_sim = pl_sim.T
    recommend = pl_sim*ptm_sparse
    recommend = np.sum(recommend, axis = 0)
    return recommend
def write_to_csv(track_score, pl_vec, playlist_id, index_of_tracks, writer):
    indices = (-track_score).argsort()
    row = [None]*501
    row[0] = playlist_id
    i = 1
    for j in range(0, 2262292):
        ind = indices[j]
        if not pl_vec[0, ind] == 0:
            row[i] = index_of_tracks[ind]
            i = i + 1
        if i > 500:
            break
    writer.writerow(row)
def write_to_csv_zero(track_score, playlist_id, index_of_tracks, writer):
    indices = (-track_score).argsort()
    row = [None]*501
    row[0] = playlist_id
    i = 1
    for j in range(0, 2262292):
        ind = indices[0, j]
        row[i] = index_of_tracks[ind]
        i = i + 1
        if i > 500:
            break
    writer.writerow(row)
def get_track_score(pl_vec, ptm_sparse, pl_sim):
    #pl_sim = cosine_similarity(ptm_sparse,pl_vec)
    pl_sim = pl_sim.T
    recommend = pl_sim*ptm_sparse
    recommend = np.sum(recommend, axis = 0)
    return recommend

In [27]:
print(len(test_data['playlists']))

50


In [28]:
u = 0
for pl in test_data['playlists']:
    if u%2 == 0:
        print(u)

    u = u + 1
    playlist_id = pl['pid']

    track_list = pl['tracks']
    track_score = np.zeros((1,c))
    doc2vec_score = np.zeros((1,c))
    if 'name' in pl:
        playlist_name = pl['name']
        doc2vec_score = get_doc2vec_score(playlist_name, doc2vec, index_of_playlist, playlists, tracks)

    if pl['num_samples'] == 0:
        track_score = top_tracks
        track_score = track_score + (pl_name_weightage*doc2vec_score)
        write_to_csv_zero(track_score, playlist_id, index_of_tracks, writer)
    else:
        pl_vec, pl_artist_vec, pl_album_vec = get_vectors(track_list, tracks, albums, artists, c, ar, al)
        
        artist_sim_score = cosine_similarity(artist_sparse,pl_artist_vec)
        album_sim_score = cosine_similarity(album_sparse, pl_album_vec)
        playlist_sim_score = cosine_similarity(ptm_sparse,pl_vec)
        
        playlist_sim_score = playlist_sim_score + (artist_boost_weightage*artist_sim_score) + (album_boost_weightage*album_sim_score)
        
        track_score = get_track_score(pl_vec, ptm_sparse, playlist_sim_score)
        #track_score = get_playlist_sim_score(pl_vec, ptm_sparse)
        
        
        #track_score = track_score + (artist_boost_weightage*artist_sim_score)
        
        
        #track_score = track_score + (album_boost_weightage*album_sim_score)
        
        #track_score = track_score + (pl_name_weightage*doc2vec_score)
        write_to_csv(track_score, pl_vec, playlist_id, index_of_tracks, writer)


0
2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
